In [1]:
import os
import pandas as pd
import tabula

In [2]:
#pip install tabula-py
from tabula import read_pdf
from PyPDF2 import PdfFileReader

In [45]:
#Now for the other folders. Transfer data is only available for 2015-2019 (which we did above)
pdf_folder = '/Users/Sou/Desktop/ABA PDF 2018/'
paths = [pdf_folder + fn for fn in os.listdir(pdf_folder) if fn.endswith('.pdf')]

In [3]:
import numpy as np

In [31]:
#Example of school name extraction on each pdf
import PyPDF2
pdfFileObj = open('/Users/Sou/Desktop/ABA PDF 2018/Std509InfoReport (100).pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pageObj = pdfReader.getPage(0)
Text= pageObj.extractText()
Text
Text.splitlines()[4] #4 corresponds to the fifth line separated

'MISSISSIPPI COLLEGE - 2018   Standard 509 Information Report'

In [3]:
def read_schoolNames(paths):
    school_names= []
    for filename in paths:
        pdfReader = PdfFileReader(open(filename, 'rb'))
        pageObj = pdfReader.getPage(0)
        Text= pageObj.extractText()
        school_text= Text.splitlines()[4]
        school_names.append(school_text)
    
    return school_names

In [48]:
school_namesIndex = read_schoolNames(paths) #success- returns a list of school names

In [49]:
def read_pdfs(paths):
    tables = []
    pdfNumPages = []
    for filename in paths:
        
        pdf = PdfFileReader(open(filename, 'rb'))
        pdfNumPages =pdf.getNumPages()
        SecondLastPage = pdfNumPages-1 #int
        LastPage = pdfNumPages #int
        LastTwoPages= list(str(SecondLastPage) +str(LastPage)) 
        transfer_tables = tabula.read_pdf(
        filename, stream= True, multiple_tables=True, 
        pages = LastTwoPages)
        
        
        tables.append(transfer_tables)
        

    return tables 

In [51]:
TransferTable2018 =[]
PageInfo2018 = read_pdfs(paths) # loop through this
for index in range(0, len(PageInfo2018)):
    try:
        TempStorage= np.array(PageInfo2018[index])
        TransferTable2018.append(pd.concat([TempStorage[-2],TempStorage[-1]])) #good
    except IndexError:
        print('Oops')
        continue
#Instead extract only the last two parts of each document
#Need to add School Name, 

In [52]:
import pickle
filename= "TransferTable-Pickled18"
output= open(filename, 'wb')
pickle.dump(TransferTable2018,output)
output.close()

In [9]:
#To unpickle:
import pickle
infile = open("/Users/Sou/Desktop/TransferTable-Pickled18",'rb')
TransferTable2018= pickle.load(infile)
infile.close()

In [53]:
#Sample of table extracted with transfer data- some have it, some don't
TransferTable2018[9] #pandas DF

,0,1
0,Schools from which Students Transferred,# of Transfer Students
1,(for schools matriculating more than 5 transfer,NaN
2,students),NaN
3,"CONNECTICUT, UNIVERSITY OF",3
4,INDIANA UNIVERSITY - BLOOMINGTON,1
5,LOYOLA UNIVERSITY-CHICAGO,1
0,"PUERTO RICO, UNIVERSITY OF",1
1,"RICHMOND, UNIVERSITY OF",1
2,SUFFOLK UNIVERSITY,2
3,"TOLEDO, UNIVERSITY OF",1


In [58]:
#To make this into a balanced table, take the school name and repeat them into one column to match
data = pd.DataFrame([])
for index in range(0, len(school_namesIndex)):
    TransferTable2= TransferTable2018[index].assign(SchoolName= school_namesIndex[index])
    #rearrange columns for better view
    cols = TransferTable2.columns.tolist()
    cols = cols[-1:] + cols[:-1] #move the last element to the first position
    TransferTable2 = TransferTable2[cols]
    data = data.append(TransferTable2)

In [59]:
data# good, remove the rest of the columns and keep the first three
data = data[['SchoolName', 0, 1]]
#Next remove rows that have NaN in them, this represent schools that don't have transfers
data = data.dropna() #drop rows with at least one NaN
#remove the undesired rows with information about races
ListofTextRemove = ['Hispanics of any race','American Indian or Alaska Native', 'Asian', 
               'Black or African American', 'Native Hawaiian or Other Pacific Islander',
              'Two or More Races', 'Total Minority', 'White', 'Nonresident Alien',
               'Race and Ethnicity Unknown','Total', 'JD1 Transfers Out','Transfers In*',
                  'Schools from which Students Transferred', '75th Percentile JD1 GPA',
                   '50th Percentile JD1 GPA', '25th Percentile JD1 GPA']
data= data.rename(columns={0:'Transfer Name'})
data= data.rename(columns={1:'Count'})
data = data[~data['Transfer Name'].isin(ListofTextRemove)]
data

,SchoolName,Transfer Name,Count
3,"BALTIMORE, UNIVERSITY OF - 2018 Standard 509 Information Report","ALABAMA, UNIVERSITY OF",1
4,"BALTIMORE, UNIVERSITY OF - 2018 Standard 509 Information Report",DISTRICT OF COLUMBIA,2
5,"BALTIMORE, UNIVERSITY OF - 2018 Standard 509 Information Report",DUQUESNE UNIVERSITY,1
0,"BALTIMORE, UNIVERSITY OF - 2018 Standard 509 Information Report",JOHN MARSHALL LAW SCHOOL,1
1,"BALTIMORE, UNIVERSITY OF - 2018 Standard 509 Information Report",LOYOLA UNIVERSITY-NEW ORLEANS,1
2,"BALTIMORE, UNIVERSITY OF - 2018 Standard 509 Information Report",VERMONT LAW SCHOOL,1
3,885,"CONNECTICUT, UNIVERSITY OF",3
4,885,INDIANA UNIVERSITY - BLOOMINGTON,1
5,885,LOYOLA UNIVERSITY-CHICAGO,1
0,885,"PUERTO RICO, UNIVERSITY OF",1


In [60]:
#Some final comestic changes:
#1) Trim the SchoolName column, everything outside of the dash
pd.set_option("display.max_rows", 1000)

#data['SchoolName'] = data['SchoolName'].str.split('-').str[0] 
# First Try: Doesn't catch everything

#2) Second Try:
#Replace dash with space in SchoolName column:
dict_replace = {'-': ' '}
data.replace(dict_replace, regex=True, inplace=True) #This applies for all columns
#remove all the text after a specific string
data["SchoolName"] = data["SchoolName"].str.split("2018", expand=True)[0]

#Turns out the name for Boston College was mis-extracted as '885'. Just replace it
data["SchoolName"] = data["SchoolName"].str.replace("885","BOSTON COLLEGE")

#Change the count column to numeric- need to aggregate
data['Count'] = data['Count'].astype(int) #Some strange pandas default that returns decimal
data2018= data
data2018

,SchoolName,Transfer Name,Count
3,"BALTIMORE, UNIVERSITY OF","ALABAMA, UNIVERSITY OF",1
4,"BALTIMORE, UNIVERSITY OF",DISTRICT OF COLUMBIA,2
5,"BALTIMORE, UNIVERSITY OF",DUQUESNE UNIVERSITY,1
0,"BALTIMORE, UNIVERSITY OF",JOHN MARSHALL LAW SCHOOL,1
1,"BALTIMORE, UNIVERSITY OF",LOYOLA UNIVERSITY NEW ORLEANS,1
2,"BALTIMORE, UNIVERSITY OF",VERMONT LAW SCHOOL,1
3,BOSTON COLLEGE,"CONNECTICUT, UNIVERSITY OF",3
4,BOSTON COLLEGE,INDIANA UNIVERSITY BLOOMINGTON,1
5,BOSTON COLLEGE,LOYOLA UNIVERSITY CHICAGO,1
0,BOSTON COLLEGE,"PUERTO RICO, UNIVERSITY OF",1


In [ ]:
#--------------------------------------------------------------------------------

In [ ]:
#For 2017- The position of the transfer table is aligned differently
#Need to adjust index accordingly

In [29]:
#Now for the other folders. Transfer data is only available for 2015-2019 (which we did above)
pdf_folder2017 = '/Users/Sou/Desktop/ABA PDF 2017/'
paths = [pdf_folder2017 + fn for fn in os.listdir(pdf_folder2017) if fn.endswith('.pdf')]

In [30]:
#Example of school name extraction on each pdf
import PyPDF2
pdfFileObj = open('/Users/Sou/Desktop/ABA PDF 2017/Std509InfoReport.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pageObj = pdfReader.getPage(0)
Text= pageObj.extractText()
Text
Text.splitlines()[4] #4 corresponds to the fifth line separated

'YALE UNIVERSITY - 2017   Standard 509 Information Report'

In [31]:
school_namesIndex2017 = read_schoolNames(paths) #success- returns a list of school names

In [32]:
#Modified version that takes into account of transfer table appearing only on last page
def read_pdfs2(paths):
    tables = []
    pdfNumPages = []
    for filename in paths:
        
        pdf = PdfFileReader(open(filename, 'rb'))
        pdfNumPages =pdf.getNumPages()
        #SecondLastPage = pdfNumPages-1 #int
        LastPage = pdfNumPages #int
        #LastTwoPages= list(str(SecondLastPage) +str(LastPage)) 
        transfer_tables = tabula.read_pdf(
        filename, lattice= True, multiple_tables=True, 
        pages = LastPage)
        
        
        tables.append(transfer_tables)
        

    return tables 

In [36]:
TransferTable2017 =[]
PageInfo2017 = read_pdfs2(paths) # loop through this
for index in range(0, len(PageInfo2017)):
    try:
        TempStorage= PageInfo2017[index][-1]
        TransferTable2017.append(TempStorage) #good
    except IndexError:
        print('Oops')
        continue
#Instead extract only the last part of each document
#Need to add School Name, 

In [37]:
import pickle
filename= "TransferTable-Pickled17"
output= open(filename, 'wb')
pickle.dump(TransferTable2017,output)
output.close()

In [18]:
#To unpickle:
import pickle
infile = open("/Users/Sou/Desktop/TransferTable-Pickled17",'rb')
TransferTable2017= pickle.load(infile)
infile.close()

In [38]:
#Sample of table extracted with transfer data- some have it, some don't
TransferTable2017[3] #pandas DF

,0,1
0,Schools from which Students Transferred\r(for schools matriculating more than 5\rtransfer students),# of\rTransfer\rStudents
1,Charleston School of Law,2
2,Florida Coastal School of Law,1
3,Indiana University Robert H McKinney\rSchool of Law,1
4,Pace University-Elisabeth Haub School of\rLaw,2
5,Santa Clara University School of Law,1
6,Stetson University College of Law,1
7,Suffolk University Law School,2
8,The Catholic University of America,1
9,The John Marshall Law School,2


In [39]:
#To make this into a balanced table, take the school name and repeat them into one column to match
data = pd.DataFrame([])
for index in range(0, len(school_namesIndex)):
    TransferTable2= TransferTable[index].assign(SchoolName= school_namesIndex[index])
    #rearrange columns for better view
    cols = TransferTable2.columns.tolist()
    cols = cols[-1:] + cols[:-1] #move the last element to the first position
    TransferTable2 = TransferTable2[cols]
    data = data.append(TransferTable2)

In [40]:
data# good, remove the rest of the columns and keep the first three
data = data[['SchoolName', 0, 1]]
#Next remove rows that have NaN in them, this represent schools that don't have transfers
data = data.dropna() #drop rows with at least one NaN
#remove the undesired rows with information about races
ListofTextRemove = ['Hispanics of any race','American Indian or Alaska Native', 'Asian', 
               'Black or African American', 'Native Hawaiian or Other Pacific Islander',
              'Two or More Races', 'Total Minority', 'White', 'Nonresident Alien',
               'Race and Ethnicity Unknown','Total', 'JD1 Transfers Out','Transfers In*',
                  'Schools from which Students Transferred', '75th Percentile JD1 GPA',
                   '50th Percentile JD1 GPA', '25th Percentile JD1 GPA', "1L Transfers Out",
                   '75th Percentile 1L GPA', '50th Percentile 1L GPA', '25th Percentile 1L GPA',
                'Schools from which Students Transferred\r(for schools matriculating more than 5\rtransfer students)'   ]
data= data.rename(columns={0:'Transfer Name'})
data= data.rename(columns={1:'Count'})
data = data[~data['Transfer Name'].isin(ListofTextRemove)]
data

,SchoolName,Transfer Name,Count
1,"ALABAMA, UNIVERSITY OF - 2017 Standard 509 Information Report",Faulkner University Thomas Jones School\rof Law,2
2,"ALABAMA, UNIVERSITY OF - 2017 Standard 509 Information Report",Michigan State University College of Law,1
3,"ALABAMA, UNIVERSITY OF - 2017 Standard 509 Information Report",Samford University-Cumberland School of\rLaw,3
4,"ALABAMA, UNIVERSITY OF - 2017 Standard 509 Information Report",Texas Southern University Thurgood\rMarshall School of Law,2
5,"ALABAMA, UNIVERSITY OF - 2017 Standard 509 Information Report",University of Kentucky College of Law,1
1,AMERICAN UNIVERSITY - 2017 Standard 509 Information Report,Charleston School of Law,2
2,AMERICAN UNIVERSITY - 2017 Standard 509 Information Report,Florida Coastal School of Law,1
3,AMERICAN UNIVERSITY - 2017 Standard 509 Information Report,Indiana University Robert H McKinney\rSchool of Law,1
4,AMERICAN UNIVERSITY - 2017 Standard 509 Information Report,Pace University-Elisabeth Haub School of\rLaw,2
5,AMERICAN UNIVERSITY - 2017 Standard 509 Information Report,Santa Clara University School of Law,1


In [24]:
pd.set_option('display.max_colwidth', -1)

In [42]:
#Some final comestic changes:
#1) Trim the SchoolName column, everything outside of the dash
pd.set_option("display.max_rows", 1000)

#data['SchoolName'] = data['SchoolName'].str.split('-').str[0] 
# First Try: Doesn't catch everything

#2) Second Try:
#Replace dash with space in SchoolName column:
dict_replace = {'-': ' '}
data.replace(dict_replace, regex=True, inplace=True) #This applies for all columns
#remove all the text after a specific string
data["SchoolName"] = data["SchoolName"].str.split("2017", expand=True)[0]

#Turns out the name for Boston College was mis-extracted as '885'. Just replace it
data["SchoolName"] = data["SchoolName"].str.replace("885","BOSTON COLLEGE")

#Change the count column to numeric- need to aggregate
#data['Count'] = data['Count'].astype(int) #Some strange pandas default that returns decimal
data2017= data
data2017

,SchoolName,Transfer Name,Count
1,"ALABAMA, UNIVERSITY OF",Faulkner University Thomas Jones School\rof Law,2
2,"ALABAMA, UNIVERSITY OF",Michigan State University College of Law,1
3,"ALABAMA, UNIVERSITY OF",Samford University Cumberland School of\rLaw,3
4,"ALABAMA, UNIVERSITY OF",Texas Southern University Thurgood\rMarshall School of Law,2
5,"ALABAMA, UNIVERSITY OF",University of Kentucky College of Law,1
1,AMERICAN UNIVERSITY,Charleston School of Law,2
2,AMERICAN UNIVERSITY,Florida Coastal School of Law,1
3,AMERICAN UNIVERSITY,Indiana University Robert H McKinney\rSchool of Law,1
4,AMERICAN UNIVERSITY,Pace University Elisabeth Haub School of\rLaw,2
5,AMERICAN UNIVERSITY,Santa Clara University School of Law,1


In [43]:
#Need to remove some more texts
#Such a strange regex string! Success!
data2017['Transfer Name'] = data2017['Transfer Name'].replace(r'\s+|\\n', ' ', regex=True)

In [44]:
data2017

,SchoolName,Transfer Name,Count
1,"ALABAMA, UNIVERSITY OF",Faulkner University Thomas Jones School of Law,2
2,"ALABAMA, UNIVERSITY OF",Michigan State University College of Law,1
3,"ALABAMA, UNIVERSITY OF",Samford University Cumberland School of Law,3
4,"ALABAMA, UNIVERSITY OF",Texas Southern University Thurgood Marshall School of Law,2
5,"ALABAMA, UNIVERSITY OF",University of Kentucky College of Law,1
1,AMERICAN UNIVERSITY,Charleston School of Law,2
2,AMERICAN UNIVERSITY,Florida Coastal School of Law,1
3,AMERICAN UNIVERSITY,Indiana University Robert H McKinney School of Law,1
4,AMERICAN UNIVERSITY,Pace University Elisabeth Haub School of Law,2
5,AMERICAN UNIVERSITY,Santa Clara University School of Law,1


In [26]:
#Using networkx
import networkx

In [64]:
#Combine 2019 +2018 + 2017 into an adjancey matrix.
data2018

,SchoolName,Transfer Name,Count
3,"BALTIMORE, UNIVERSITY OF","ALABAMA, UNIVERSITY OF",1
4,"BALTIMORE, UNIVERSITY OF",DISTRICT OF COLUMBIA,2
5,"BALTIMORE, UNIVERSITY OF",DUQUESNE UNIVERSITY,1
0,"BALTIMORE, UNIVERSITY OF",JOHN MARSHALL LAW SCHOOL,1
1,"BALTIMORE, UNIVERSITY OF",LOYOLA UNIVERSITY NEW ORLEANS,1
2,"BALTIMORE, UNIVERSITY OF",VERMONT LAW SCHOOL,1
3,BOSTON COLLEGE,"CONNECTICUT, UNIVERSITY OF",3
4,BOSTON COLLEGE,INDIANA UNIVERSITY BLOOMINGTON,1
5,BOSTON COLLEGE,LOYOLA UNIVERSITY CHICAGO,1
0,BOSTON COLLEGE,"PUERTO RICO, UNIVERSITY OF",1


In [97]:
#2019
pdf_folder2019 = '/Users/Sou/Desktop/ABA PDF 2019/'
paths = [pdf_folder2019 + fn for fn in os.listdir(pdf_folder2019) if fn.endswith('.pdf')]

In [98]:
#Example of school name extraction on each pdf
import PyPDF2
pdfFileObj = open('/Users/Sou/Desktop/ABA PDF 2019/Std509InfoReport (1).pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pageObj = pdfReader.getPage(0)
Text= pageObj.extractText()
Text
Text.splitlines()[4] #4 corresponds to the fifth line separated

'AKRON, UNIVERSITY OF - 2019   Standard 509 Information Report'

In [99]:
school_namesIndex2019 = read_schoolNames(paths) #success- returns a list of school names
school_namesIndex2019

['AKRON, UNIVERSITY OF - 2019   Standard 509 Information Report',
 'ARKANSAS, LITTLE ROCK, UNIVERSITY OF - 2019   Standard 509 Information Report',
 'MICHIGAN, UNIVERSITY OF - 2019   Standard 509 Information Report',
 "ATLANTA'S JOHN MARSHALL LAW SCHOOL - 2019   Standard 509 Information ",
 'AVE MARIA SCHOOL OF LAW - 2019   Standard 509 Information Report',
 'BALTIMORE, UNIVERSITY OF - 2019   Standard 509 Information Report',
 'BARRY UNIVERSITY - 2019   Standard 509 Information Report',
 'BAYLOR UNIVERSITY - 2019   Standard 509 Information Report',
 'BELMONT UNIVERSITY - 2019   Standard 509 Information Report',
 '885',
 'BOSTON UNIVERSITY - 2019   Standard 509 Information Report',
 'BRIGHAM YOUNG UNIVERSITY - 2019   Standard 509 Information Report',
 'ALABAMA, UNIVERSITY OF - 2019   Standard 509 Information Report',
 'BROOKLYN LAW SCHOOL - 2019   Standard 509 Information Report',
 'CALIFORNIA WESTERN SCHOOL OF LAW - 2019   Standard 509 Information Report',
 'CALIFORNIA-BERKELEY, UNIVER

In [100]:
#Modified version that takes into account of transfer table appearing only on last page
def read_pdfs3(paths):
    tables = []
    pdfNumPages = []
    for filename in paths:
        
        pdf = PdfFileReader(open(filename, 'rb'))
        pdfNumPages =pdf.getNumPages()
        #SecondLastPage = pdfNumPages-1 #int
        LastPage = pdfNumPages #int
        #LastTwoPages= list(str(SecondLastPage) +str(LastPage)) 
        transfer_tables = tabula.read_pdf(
        filename, lattice= True, multiple_tables=True, 
        pages = LastPage)
        
        
        tables.append(transfer_tables)
        

    return tables 

In [101]:
TransferTable2019 =[]
PageInfo2019 = read_pdfs3(paths) # loop through this
for index in range(0, len(PageInfo2019)):
    try:
        TempStorage= PageInfo2019[index][-1]
        TransferTable2019.append(TempStorage) #good
    except IndexError:
        print('Oops')
        continue

#Two files in this folder cannot be found!

Oops
Oops


In [102]:
import pickle
filename= "TransferTable-Pickled19"
output= open(filename, 'wb')
pickle.dump(TransferTable2019,output)
output.close()

In [ ]:
#To unpickle:
import pickle
infile = open("/Users/Sou/Desktop/TransferTable-Pickled19",'rb')
TransferTable2019= pickle.load(infile)
infile.close()

In [112]:
TransferTable2019[9] #just to check- recall some have it and some don't

,0,1,2
0,LOYOLA MARYMOUNT UNIVERSITY-LOS ANGELES,1,NaN
1,MICHIGAN STATE UNIVERSITY,1,NaN
2,ROGER WILLIAMS UNIVERSITY,1,NaN
3,SUFFOLK UNIVERSITY,1,NaN
4,TULANE UNIVERSITY,1,NaN
5,WESTERN NEW ENGLAND UNIVERSITY,1,NaN


In [128]:
data = pd.DataFrame([])
for index in range(0, len(school_namesIndex2019)-2): #reduce by two since two files couldn't be identified
    TransferTable2_2019= TransferTable2019[index].assign(SchoolName= school_namesIndex2019[index])
    #rearrange columns for better view
    cols = TransferTable2_2019.columns.tolist()
    cols = cols[-1:] + cols[:-1] #move the last element to the first position
    TransferTable2_2019 = TransferTable2_2019[cols]
    data = data.append(TransferTable2_2019)

In [129]:
data# good, remove the rest of the columns and keep the first three
data = data[['SchoolName', 0, 1]]
#Next remove rows that have NaN in them, this represent schools that don't have transfers
data = data.dropna() #drop rows with at least one NaN
#remove the undesired rows with information about races
ListofTextRemove = ['Hispanics of any race','American Indian or Alaska Native', 'Asian', 
               'Black or African American', 'Native Hawaiian or Other Pacific Islander',
              'Two or More Races', 'Total Minority', 'White', 'Nonresident Alien',
               'Race and Ethnicity Unknown','Total', 'JD1 Transfers Out','Transfers In*',
                  'Schools from which Students Transferred', '75th Percentile JD1 GPA',
                   '50th Percentile JD1 GPA', '25th Percentile JD1 GPA', "1L Transfers Out",
                   '75th Percentile 1L GPA', '50th Percentile 1L GPA', '25th Percentile 1L GPA',
                'Schools from which Students Transferred\r(for schools matriculating more than 5\rtransfer students)'   ]
data= data.rename(columns={0:'Transfer Name'})
data= data.rename(columns={1:'Count'})
data = data[~data['Transfer Name'].isin(ListofTextRemove)]
data

,SchoolName,Transfer Name,Count
0,"MICHIGAN, UNIVERSITY OF - 2019 Standard 509 Information Report",EMORY UNIVERSITY,1
1,"MICHIGAN, UNIVERSITY OF - 2019 Standard 509 Information Report",FLORIDA STATE UNIVERSITY,1
2,"MICHIGAN, UNIVERSITY OF - 2019 Standard 509 Information Report",GEORGE WASHINGTON UNIVERSITY,2
3,"MICHIGAN, UNIVERSITY OF - 2019 Standard 509 Information Report",MICHIGAN STATE UNIVERSITY,4
4,"MICHIGAN, UNIVERSITY OF - 2019 Standard 509 Information Report","MISSISSIPPI, UNIVERSITY OF",1
5,"MICHIGAN, UNIVERSITY OF - 2019 Standard 509 Information Report",OHIO STATE UNIVERSITY,1
6,"MICHIGAN, UNIVERSITY OF - 2019 Standard 509 Information Report",WAYNE STATE UNIVERSITY,1
0,885,LOYOLA MARYMOUNT UNIVERSITY-LOS ANGELES,1
1,885,MICHIGAN STATE UNIVERSITY,1
2,885,ROGER WILLIAMS UNIVERSITY,1


In [116]:
pd.set_option('display.max_colwidth', -1)

In [136]:
#Some final comestic changes:
#1) Trim the SchoolName column, everything outside of the dash
pd.set_option("display.max_rows", 1000)

#data['SchoolName'] = data['SchoolName'].str.split('-').str[0] 
# First Try: Doesn't catch everything

#2) Second Try:
#Replace dash with space in SchoolName column:
dict_replace = {'-': ' '}
data.replace(dict_replace, regex=True, inplace=True) #This applies for all columns
#remove all the text after a specific string
data["SchoolName"] = data["SchoolName"].str.split("2019", expand=True)[0]

#Turns out the name for Boston College was mis-extracted as '885'. Just replace it
data["SchoolName"] = data["SchoolName"].str.replace("885","BOSTON COLLEGE")

#Change the count column to numeric- need to aggregate
#data['Count'] = data['Count'].astype(int) #Some strange pandas default that returns decimal
data2019= data
data2019

,SchoolName,Transfer Name,Count
0,"MICHIGAN, UNIVERSITY OF",EMORY UNIVERSITY,1
1,"MICHIGAN, UNIVERSITY OF",FLORIDA STATE UNIVERSITY,1
2,"MICHIGAN, UNIVERSITY OF",GEORGE WASHINGTON UNIVERSITY,2
3,"MICHIGAN, UNIVERSITY OF",MICHIGAN STATE UNIVERSITY,4
4,"MICHIGAN, UNIVERSITY OF","MISSISSIPPI, UNIVERSITY OF",1
5,"MICHIGAN, UNIVERSITY OF",OHIO STATE UNIVERSITY,1
6,"MICHIGAN, UNIVERSITY OF",WAYNE STATE UNIVERSITY,1
0,BOSTON COLLEGE,LOYOLA MARYMOUNT UNIVERSITY LOS ANGELES,1
1,BOSTON COLLEGE,MICHIGAN STATE UNIVERSITY,1
2,BOSTON COLLEGE,ROGER WILLIAMS UNIVERSITY,1


In [138]:
#Need to remove some more texts
#Such a strange regex string! Success!
data2019['Transfer Name'] = data2019['Transfer Name'].replace(r'\s+|\\n', ' ', regex=True)
data2019

,SchoolName,Transfer Name,Count
0,"MICHIGAN, UNIVERSITY OF",EMORY UNIVERSITY,1
1,"MICHIGAN, UNIVERSITY OF",FLORIDA STATE UNIVERSITY,1
2,"MICHIGAN, UNIVERSITY OF",GEORGE WASHINGTON UNIVERSITY,2
3,"MICHIGAN, UNIVERSITY OF",MICHIGAN STATE UNIVERSITY,4
4,"MICHIGAN, UNIVERSITY OF","MISSISSIPPI, UNIVERSITY OF",1
5,"MICHIGAN, UNIVERSITY OF",OHIO STATE UNIVERSITY,1
6,"MICHIGAN, UNIVERSITY OF",WAYNE STATE UNIVERSITY,1
0,BOSTON COLLEGE,LOYOLA MARYMOUNT UNIVERSITY LOS ANGELES,1
1,BOSTON COLLEGE,MICHIGAN STATE UNIVERSITY,1
2,BOSTON COLLEGE,ROGER WILLIAMS UNIVERSITY,1


In [139]:
#Using networkx
edgeList = data2018.values.tolist() + data2017.values.tolist() + data2019.values.tolist()
edgeList #list object

[['BALTIMORE, UNIVERSITY OF   ', 'ALABAMA, UNIVERSITY OF', 1],
 ['BALTIMORE, UNIVERSITY OF   ', 'DISTRICT OF COLUMBIA', 2],
 ['BALTIMORE, UNIVERSITY OF   ', 'DUQUESNE UNIVERSITY', 1],
 ['BALTIMORE, UNIVERSITY OF   ', 'JOHN MARSHALL LAW SCHOOL', 1],
 ['BALTIMORE, UNIVERSITY OF   ', 'LOYOLA UNIVERSITY NEW ORLEANS', 1],
 ['BALTIMORE, UNIVERSITY OF   ', 'VERMONT LAW SCHOOL', 1],
 ['BOSTON COLLEGE', 'CONNECTICUT, UNIVERSITY OF', 3],
 ['BOSTON COLLEGE', 'INDIANA UNIVERSITY   BLOOMINGTON', 1],
 ['BOSTON COLLEGE', 'LOYOLA UNIVERSITY CHICAGO', 1],
 ['BOSTON COLLEGE', 'PUERTO RICO, UNIVERSITY OF', 1],
 ['BOSTON COLLEGE', 'RICHMOND, UNIVERSITY OF', 1],
 ['BOSTON COLLEGE', 'SUFFOLK UNIVERSITY', 2],
 ['BOSTON COLLEGE', 'TOLEDO, UNIVERSITY OF', 1],
 ['ALABAMA, UNIVERSITY OF   ', 'FAULKNER UNIVERSITY', 2],
 ['ALABAMA, UNIVERSITY OF   ', 'FLORIDA INTERNATIONAL UNIVERSITY', 1],
 ['ALABAMA, UNIVERSITY OF   ', 'FLORIDA, UNIVERSITY OF', 1],
 ['ALABAMA, UNIVERSITY OF   ', 'MEMPHIS, UNIVERSITY OF', 1],
 ['A